In [1]:
from osgeo import gdal
import sys
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
##########################################
### clip raster by study area (vector) ###
##########################################

In [3]:
path_raster = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march/S2A_20180322/B02_10.jp2"
path_vector = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/temp/study_area/boundry.shp"
path_out = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march_clip/S2A_20180322/B02_10.jp2"

In [4]:
path_in = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march/"
path_out = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march_clip/"

In [34]:
dirs_rast = sorted(os.listdir(path_in))

for dir_rast in dirs_rast[2:]:
    print('*********************')
    print(dir_rast)
    print(sorted(os.listdir(path_in+dir_rast))[:10])
    #poi = extract_values(poi, path_raster+dir_rast+'/')

*********************
S2A_20180322
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180421
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180513
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180617
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180702
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.jp2', 'B08A_20.jp2', 'B08_10.jp2', 'B11_20.jp2', 'B12_20.jp2']
*********************
S2A_20180720
['B02_10.jp2', 'B03_10.jp2', 'B04_10.jp2', 'B05_20.jp2', 'B06_20.jp2', 'B07_20.j

In [35]:
# load vector - study area boundries
vect = gpd.read_file(path_vector)
# get extend
extend = vect.loc[0,'geometry'].bounds

In [36]:
extend

(602197.2084267415, 5330943.329159929, 647517.2084267415, 5373373.329159929)

In [39]:
dir_dates = sorted(os.listdir(path_in))

# loop over dates
for dir_date in dir_dates[2:]:
    print('*********************')
    print(dir_date)
    dir_bands = sorted(os.listdir(path+dir_date))[:10]
    
    # loop over bands
    for dir_band in dir_bands:
        rast_in = "{}{}/{}".format(path_in,dir_date,dir_band)
        rast_out = "{}{}/{}".format(path_out,dir_date,dir_band)
        
        # construct bash command
        bash_command = "gdal_translate -projwin {} {} {} {} -a_nodata 0.0 -of GTiff {} {}".format(extend[0], extend[3], extend[2], extend[1], rast_in, rast_out)
        # execute command
        os.system(bash_command)

*********************
S2A_20180322
*********************
S2A_20180421
*********************
S2A_20180513
*********************
S2A_20180617
*********************
S2A_20180702
*********************
S2A_20180720
*********************
S2A_20180809
*********************
S2A_20180821
*********************
S2A_20180920


In [ ]:
###############################

In [4]:
def tif2array(file_path, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    inputs:
        file_path (str) : file path of the input GeoTiff file
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive
    """
    dataset = gdal.Open(file_path, gdal.GA_ReadOnly)

    if dataset is None:
        return None

    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)

    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)

    #image = image[2:-2,2:-2,:]

    return image

In [44]:
# create array for storing rearranged data
data = np.zeros([19229276,90], np.uint16)
i = 0

# get directories of aqusition dates
dir_dates = sorted(os.listdir(path_out))

# loop over dates
for dir_date in dir_dates:
    print('*********************')
    print(i, dir_date)
    # get filenames of bands
    dir_bands = sorted(os.listdir(path_out+dir_date))
    
    # loop over bands
    for dir_band in dir_bands:
        # create path
        rast = "{}{}/{}".format(path_out,dir_date,dir_band)
        # load and convert to numpy array
        x = tif2array(rast, dtype=np.uint16)
        # if resolution is 20m -> rescale to 10m
        start = dir_band.find('_')
        if dir_band[start+1:start+3] == '20':
            x = np.repeat(x, 2, axis=0)
            x = np.repeat(x, 2, axis=1)
            x = x[:-1,:]
        data[:,i] = x.flatten()
        
        i += 1
        del x
        # construct bash command
    #    bash_command = "gdal_translate -projwin {} {} {} {} -a_nodata 0.0 -of GTiff {} {}".format(extend[0], extend[3], extend[2], extend[1], rast_in, rast_out)
        # execute command
    #    os.system(bash_command)

*********************
0 S2A_20180322
*********************
10 S2A_20180421
*********************
20 S2A_20180513
*********************
30 S2A_20180617
*********************
40 S2A_20180702
*********************
50 S2A_20180720
*********************
60 S2A_20180809
*********************
70 S2A_20180821
*********************
80 S2A_20180920


In [49]:
data

array([[ 405,  535,  727, ..., 3600, 1919, 1060],
       [ 433,  562,  754, ..., 3419, 1919, 1060],
       [ 461,  589,  770, ..., 3323, 1950, 1127],
       ...,
       [ 354,  378,  455, ..., 2879, 1499,  660],
       [ 398,  394,  507, ..., 2974, 1535,  665],
       [ 344,  346,  431, ..., 2041, 1535,  665]], dtype=uint16)

In [48]:
data.tofile('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/data/class_data.csv', sep = ',')

In [50]:
np.save('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/data/class_data', data)

In [ ]:
#######################
## convert to geotiff #
#######################

In [3]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [4]:
def array2raster(newRasterfn, dataset, array, dtype, offsetX=0, offsetY=0):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform()
    originX += offsetX
    originY += offsetY

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [5]:
pred = np.load('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/data/pred_data.npy')

In [6]:
pred.shape

(4243, 4532)

In [7]:
path_in = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/march_clip/S2A_20180322/B02_10.jp2'
path_out = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/data/pred_img.tiff'

In [9]:
# create output path
#path_out = '{}pred{}'.format(path, path_in[path_in.rfind('_'):])
# open coresponding ortho file
dataset = gdal.Open(path_in, gdal.GA_ReadOnly)
# create rater (GeoTiff) file out of numpy array (prediction) and coresponding ortho file information  
array2raster(newRasterfn=path_out, dataset=dataset, array=pred, dtype='Byte')

In [ ]:
### vectorize ###

In [10]:
vector_file = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/data/pred_img.shp'
# create vector (polygons) out of raster
cmd = 'gdal_polygonize.py -8 {} -b 1 -f "ESRI Shapefile" {} output crop'.format(path_out, vector_file)
# execute bash command
os.system(cmd)

0

In [ ]:
# create numpy array

In [ ]:
# classify

In [ ]:
# create image

In [16]:
vector_file = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/temp/classified/fiels_boundry.shp"
raster_in = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s2_raster/data/pred_img_crs.tiff"
raster_out = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/temp/classified/pred_img_clip.tiff"

In [17]:
cmd = 'gdalwarp -of GTiff -cutline {} -cl fiels_boundry -crop_to_cutline {} {}'.format(vector_file, raster_in, raster_out, )
# execute bash command
os.system(cmd)

256